# Note: This notebook only runs on local cluster right now because the EMR cluster doesnt support packages like Voila and wordcloud. We have done this as an exploratory analysis just for experimentation. Hence it is unpotimized. Please dont grade based on this :D

## Use the following instructions to install voila and use it. 

On your jupyter notebook, run

!pip install voila

Setup the voila extension to Jupyter notebook using

!jupyter serverextension enable --sys-prefix voila

Restart the Jupyter notebook. you will be able to find the voila extension on the menu bar. Click on it. It will redirect you to a version of your notebook which only shows markdowns and interactive ipython widgets.

In [1]:
import pandas as pd
import numpy as np
import pyspark
import os
import re
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import widgets, interactive
import ipywidgets as widgets
%config InlineBackend.figure_format = 'retina'
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import time
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")


t_start = time.time()

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.hadoop:hadoop-aws:3.3.1" pyspark-shell'
sc = pyspark.SparkContext.getOrCreate()

21/12/08 21:59:26 WARN Utils: Your hostname, Ashwanis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.202 instead (on interface en0)
21/12/08 21:59:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/ashwanirajan/opt/anaconda3/envs/DistComputing/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ashwanirajan/.ivy2/cache
The jars for the packages stored in: /Users/ashwanirajan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-44fb45fb-5ff0-47d1-847f-130b44974577;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 354ms :: artifacts dl 9ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------

In [3]:
sc._jsc.hadoopConfiguration().set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

In [5]:
sc._jsc.hadoopConfiguration().set('fs.s3a.access.key', os.environ['AWS_ACCESS_KEY_ID'])

In [6]:
sc._jsc.hadoopConfiguration().set('fs.s3a.secret.key', os.environ['AWS_SECRET_ACCESS_KEY'])

In [7]:
file = 's3://millionsongs-data/new_g4g.csv'

In [8]:
conf = (
 pyspark.SparkConf()
 .set('spark.driver.memory ','10g')
 .set('spark.executor.memory ','10g')
 .setAppName('HW3')
)

In [9]:
sc = pyspark.SparkContext.getOrCreate(conf = conf)
sc.setLogLevel("ERROR")

In [10]:
cols = ['get_analysis_sample_rate','get_artist_7digitalid','get_artist_familiarity',
 'get_artist_hotttnesss','get_artist_id','get_artist_latitude','get_artist_location',
'get_artist_longitude','get_artist_mbid','get_artist_mbtags','get_artist_mbtags_count',
 'get_artist_name','get_artist_playmeid','get_artist_terms','get_artist_terms_freq','get_artist_terms_weight',
 'get_audio_md5','get_bars_confidence','get_bars_start','get_beats_confidence','get_beats_start','get_danceability',
 'get_duration','get_end_of_fade_in','get_energy','get_key','get_key_confidence','get_loudness','get_mode',
 'get_mode_confidence','get_num_songs','get_release','get_release_7digitalid','get_sections_confidence',
 'get_sections_start','get_segments_confidence','get_segments_loudness_max','get_segments_loudness_max_time',
 'get_segments_loudness_start','get_segments_pitches','get_segments_start','get_segments_timbre','get_similar_artists',
 'get_song_hotttnesss','get_song_id','get_start_of_fade_out','get_tatums_confidence','get_tatums_start',
 'get_tempo','get_time_signature','get_time_signature_confidence','get_title','get_track_7digitalid',
'get_track_id','get_year']

In [11]:
input_rdd = sc.textFile(file)
table = input_rdd.map(lambda x: x.split('\t'))
header = table.take(1)
table_body = table.filter(lambda x : x != header[0])
column_index = {header[0][i]:i for i in range(len(header[0]))}

In [12]:
artist_rdd = table_body.filter(lambda x : x[column_index['get_artist_familiarity']]!= '').filter(lambda x : 
                        x[column_index['get_artist_hotttnesss']]!= '').map(lambda x: 
                       (x[column_index['get_artist_id']],
                        [x[column_index['get_artist_name']],
                        float(x[column_index['get_artist_familiarity']]),
                        float(x[column_index['get_artist_hotttnesss']]),
                        x[column_index['get_artist_mbtags']],
                        x[column_index['get_artist_mbtags_count']],
                        x[column_index['get_artist_terms']],
                        x[column_index['get_artist_terms_freq']],
                        x[column_index['get_similar_artists']],
                        x[column_index['get_energy']]]))

artist_rdd_names = artist_rdd.map(lambda x: (x[1][0], 
                    x[1][2])).groupByKey().mapValues(
    lambda x: sum(x)/len(x)).sortBy(lambda x :x[1], ascending=False)
hottest_artists = artist_rdd_names.map(lambda x: [x[0],x[1]]).take(30)
hot_artists_df = pd.DataFrame(hottest_artists, columns = ['artist', 'popularity'])
hottest_artists_name = [x[0] for x in hottest_artists]

In [13]:
artist_rdd_terms = table_body.map(lambda x: (x[column_index['get_artist_name']], 
                                             x[column_index['get_artist_terms']] #,
                                              #x[column_index['get_artist_terms_freq']
                        )).filter(lambda x : x[0] in hottest_artists_name)

artist_rdd_term_weights = table_body.map(lambda x: (x[column_index['get_artist_name']], 
                                             x[column_index['get_artist_terms_freq']]
                        )).filter(lambda x : x[0] in hottest_artists_name)

artist_rdd_term_weights_float = artist_rdd_term_weights.groupByKey().mapValues(
    lambda x: list(x)).map(
    lambda x : (x[0], [float(l) for l in x[1][0].replace('[', '').replace(']', '').split()]))

artist_rdd_term_weights_float = artist_rdd_term_weights.groupByKey().mapValues(lambda x: list(x)).map(
    lambda x : (x[0], [float(l) for l in x[1][0].replace('[', '').replace(']', '').split()]))

artist_rdd_terms_frame = artist_rdd_terms.groupByKey().mapValues(lambda x: list(x)).reduceByKey(lambda x,y: x+y).map(
    lambda x : (x[0], x[1][0])).mapValues(lambda x :re.findall(r"'(.*?)'", x, re.DOTALL))

artist_terms_dataset = artist_rdd_terms_frame.join(artist_rdd_term_weights_float)
df_artist = dict(artist_terms_dataset.collect())

In [14]:
df_artist_terms_dict = {key: {df_artist[key][0][i]: df_artist[key][1][i] for i in 
                              range(len(df_artist[key][1]))} for key in df_artist}

In [15]:
similar_artists = table_body.map(lambda x: (x[column_index['get_artist_id']],
                          [x[column_index['get_artist_name']], 
                           x[column_index['get_similar_artists']]]))

artists = similar_artists.map(lambda x :(x[0], x[1][0]))
ArtistDict = artists.collectAsMap()

In [16]:
def get_list_of_artists(str_lst):
    str_lst = str_lst.replace('[', '').replace(']', '').replace("'", "")
    ids = str_lst.split()
    id_names = []
    for i in ids:
        if i in ArtistDict:
            id_names.append(ArtistDict[i])
    return id_names

In [17]:
similar_artists_names = similar_artists.map(lambda x: (x[1][0], get_list_of_artists(x[1][1])))
SimilarArtistDict = similar_artists_names.collectAsMap()

## Related Tags and Similar Artists

In [18]:

def wordcloud_fn(artist_name):
    fig = plt.figure(figsize=(10, 20))
    wordcloud = WordCloud(width=1800,
                          height=1400,
                          background_color="white",
                          max_words=1000,
                          random_state=1,
                          relative_scaling=0.25)

    wordcloud.fit_words(df_artist_terms_dict[artist_name])

    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()
    
    print(f'Similar Artists to {artist_name}: ')
    print()
    for i in range(10):
        print(SimilarArtistDict[artist_name][i])

In [19]:
dropdown = widgets.Dropdown(
    options=list(df_artist_terms_dict.keys()),
    value='Coldplay',
    description='Number:',
    disabled=False,
)

In [20]:
# Initiating Button to generate Population distribution and plot it.
button_pop = widgets.Button(
    description='Get Wordcloud and Similar Artists',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    layout=widgets.Layout(width='30%', height='50px')
)

In [21]:
display(dropdown)

Dropdown(description='Number:', options=('Coldplay', 'Maroon 5 / Cross', 'The Black Eyed Peas', 'Usher', 'Emin…

In [22]:
out_pop = widgets.Output()  # Initiate output of Button


def create_pop(b):
    """
    This function is triggered when the create population button is pressed.
    The population_dist function is called and the plots are presented
    """
    with out_pop:
        out_pop.clear_output(True)
        wordcloud_fn(dropdown.value)


button_pop.on_click(create_pop)

# Displays the Create population button
display(button_pop)

# displaying the output plot
with out_pop:
    wordcloud_fn(dropdown.value)
print('\n\n\n')
out_pop


Button(button_style='info', description='Get Wordcloud and Similar Artists', layout=Layout(height='50px', widt…

Output()